# DLL Load via LSASS
Detects a method to load DLL via LSASS process using an undocumented Registry key

## Rule Content
```
- title: DLL Load via LSASS
  id: b3503044-60ce-4bf4-bbcb-e3db98788823
  status: experimental
  description: Detects a method to load DLL via LSASS process using an undocumented
    Registry key
  author: Florian Roth
  date: 2019/10/16
  references:
  - https://blog.xpnsec.com/exploring-mimikatz-part-1/
  - https://twitter.com/SBousseaden/status/1183745981189427200
  logsource:
    product: windows
    service: sysmon
    category: null
  detection:
    selection:
      EventID:
      - 12
      - 13
      TargetObject:
      - '*\CurrentControlSet\Services\NTDS\DirectoryServiceExtPt*'
      - '*\CurrentControlSet\Services\NTDS\LsaDbExtPt*'
    condition: selection
  tags:
  - attack.execution
  - attack.t1177
  falsepositives:
  - Unknown
  level: high

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-endpoint-winevent-sysmon-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='(event_id:("12" OR "13") AND registry_key_path.keyword:(*\\CurrentControlSet\\Services\\NTDS\\DirectoryServiceExtPt* OR *\\CurrentControlSet\\Services\\NTDS\\LsaDbExtPt*))')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()